In [13]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import csv
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from collections import Counter
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import OneHotEncoder



In [2]:
data = pd.read_csv('/Users/sanyamthakur/Documents/GitHub/Data_Science/datasets/tested.csv')


In [3]:
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
data.shape


(418, 12)

In [5]:
data.dtypes

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

<h5>Dropping the unwanted columns. <i>id, Fare, Cabin</i> seems unwanted columns, so let's remove them.</h5>

In [6]:
data = data.drop(['Name','Fare','Cabin','Ticket'],axis=1)


In [7]:
#c create a new column called 'Family' in the DataFrame 'df'. 
# This column represents the family size of each passenger on the Titanic.
data['family'] = data['Parch']+1+data['SibSp']
data  = data.drop(['Parch','SibSp'],axis=1)

<h3>Pre Processing the data</h3>

In [8]:
#filling the age columns
data['Age'] = data['Age'].fillna(data['Age'].median())

In [9]:
data['Embarked'].value_counts()

Embarked
S    270
C    102
Q     46
Name: count, dtype: int64

In [10]:
data['Embarked'] = data['Embarked'].fillna('S')


OneHotEncoding

In [11]:
onehot_encoder = OneHotEncoder(sparse=False)
onehot = onehot_encoder.fit_transform(data[['Embarked','Sex']])
feature_names = onehot_encoder.get_feature_names_out(['Embarked', 'Sex'])
forcont = pd.DataFrame(onehot,columns=feature_names)
data = pd.concat([data.drop(['Embarked','Sex'],axis=1),forcont],axis=1)


In [12]:
data

,PassengerId,Survived,Pclass,Age,family,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male
0,892,0,3,34.5,1,0.0,1.0,0.0,0.0,1.0
1,893,1,3,47.0,2,0.0,0.0,1.0,1.0,0.0
2,894,0,2,62.0,1,0.0,1.0,0.0,0.0,1.0
3,895,0,3,27.0,1,0.0,0.0,1.0,0.0,1.0
4,896,1,3,22.0,3,0.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
413,1305,0,3,27.0,1,0.0,0.0,1.0,0.0,1.0
414,1306,1,1,39.0,1,1.0,0.0,0.0,1.0,0.0
415,1307,0,3,38.5,1,0.0,0.0,1.0,0.0,1.0
416,1308,0,3,27.0,1,0.0,0.0,1.0,0.0,1.0


Finding optimal K using cross validation (simple cross validation and KFold cross validation)